In [1]:
import os

import copy as cp
import numpy as np
import matplotlib.pyplot as plt
import getdist as gd
import getdist.plots as gdp

%matplotlib inline

from lyaemu import likelihood as lyl
from lyaemu import lyman_data as lyd


/share/apps/anaconda/python3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/keir/Software/lya_emulator/lyaemu/SimulationRunner/SimulationRunner/cambpower.py:8: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/share/apps/anaconda/python3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/share/apps/anaconda/python3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/share/apps/an

In [2]:
basedir = '/share/data2/keir/Simulations/nCDM_emulator_512'
emulator_json = 'emulator_params_TDR_u0_original.json'
use_measured_params = True
redshift_dependent_params = True

lyman_data_instance = lyd.BoeraData()
z = lyman_data_instance.redshifts_unique[::-1]


likelihood_instance = lyl.UltraLightAxionLikelihoodClass(basedir, mean_flux='free_high_z', max_z=np.max(z),
                                                         redshifts=z, pixel_resolution_km_s=1., emulator_class='nCDM',
                                                         emulator_json_file=emulator_json,
                                                         use_measured_parameters=use_measured_params,
                                                         redshift_dependent_parameters=redshift_dependent_params,
                                                         data_class='Boera',
                                                         dark_matter_model=lyl.ultra_light_axion_numerical_model)

likelihood_instance.data_fluxpower = likelihood_instance.lyman_data_flux_power[::-1].flatten()


Beginning to generate emulator at 2020-03-26 19:16:58.829377
k_max_emulated_h_Mpc = 28.093018599498517 0.19952623149688797 4.95 0.33
dpvals = [[0.73805814]
 [0.79181238]
 [0.84556662]
 [0.89932086]
 [0.95307511]
 [1.00682935]
 [1.06058359]
 [1.11433784]
 [1.16809208]
 [1.22184632]]
nuggets = [0.         0.00096758 0.00193515 0.00290273 0.00387031 0.00483788
 0.00580546 0.00677303 0.00774061 0.00870819 0.00967576 0.01064334
 0.01161092 0.01257849 0.01354607 0.01451365 0.01548122 0.0164488
 0.01741637 0.01838395 0.01935153 0.0203191  0.02128668 0.02225426
 0.02322183 0.02418941 0.02515699 0.02612456 0.02709214 0.02805971
 0.02902729 0.02999487 0.03096244 0.03193002 0.0328976  0.03386517
 0.03483275 0.03580033 0.0367679  0.03773548 0.03870306 0.03967063
 0.04063821 0.04160578 0.04257336 0.04354094 0.04450851 0.04547609
 0.04644367 0.04741124]
mean_flux = [[0.27388422 0.36723956 0.45885946]
 [0.24923206 0.34140027 0.43354996]
 [0.22679882 0.31737906 0.40963647]
 [0.20638478 0.29504799 0.38

In [3]:
#Prior function
prior_gaussian_param_names = np.array(['tau0_0', 'tau0_1', 'tau0_2', 'ns', 'As', 'omega_m', 'T_0_z_5.0', 'T_0_z_4.6',
                                      'T_0_z_4.2'])
prior_gaussian_mean = np.array([1., 1., 1., 0.9635, 1.8296e-9, 0.3209, 8022., 7651., 8673.])
prior_gaussian_std = np.array([0.05, 0.05, 0.05, 0.0057, 0.030 * 1.e-9, 0.00001, 3000., 3000., 3000.])
prior_gaussian = lambda params: likelihood_instance.log_gaussian_prior(params, prior_gaussian_param_names,
                                                                       prior_gaussian_mean, prior_gaussian_std)

prior_max_jump_param_names = np.array(['T_0', 'u_0'])
prior_max_jumps = np.array([5000., 10.])
prior_max_jump = lambda params: likelihood_instance.log_redshift_prior(params, prior_max_jump_param_names,
                                                                       prior_max_jumps)

prior_convex_hull_param_names = [['T_0_z_5.0', 'u_0_z_5.0'], ['T_0_z_4.6', 'u_0_z_4.6'], ['T_0_z_4.2', 'u_0_z_4.2']]
prior_convex_hull = lambda params: likelihood_instance.log_convex_hull_prior(params, prior_convex_hull_param_names)

prior_functions = [prior_gaussian, prior_max_jump, prior_convex_hull]


In [4]:
#Test high likelihood
test_params = np.array([0.9635, 1.8296e-9, 0.3209, 8022., 7651., 8673., 1.44, 1.48, 1.45, 4.56, 6.11, 7.24, -19.1])
test_params_mf = np.concatenate((np.array([0.77, 0.78, 0.93]), test_params))
test_params_mf.shape


(16,)

In [ ]:
#Test low likelihood
test_params = np.array([0.975, 1.75e-9, 0.3, 13000., 13000., 13000., 1., 1., 1., 20., 20., 20., -21.])
test_params_mf = np.concatenate((np.ones(3) * 1., test_params))
test_params_mf.shape


In [ ]:
#Test BO functions
log_posterior = likelihood_instance.log_posterior_marginalised_mean_flux(test_params, prior_functions=prior_functions)
print('log posterior marginalised =', log_posterior)


In [ ]:
#Test BO functions [low likelihood]
log_posterior = likelihood_instance.log_posterior_marginalised_mean_flux(test_params, prior_functions=prior_functions)
print('log posterior marginalised =', log_posterior)


In [ ]:
#Test log posterior
log_posterior = likelihood_instance.log_posterior(test_params_mf, prior_functions=prior_functions)
print('log posterior =', log_posterior)


In [ ]:
#Test log posterior [low likelihood]
log_posterior = likelihood_instance.log_posterior(test_params_mf, prior_functions=prior_functions)
print('log posterior =', log_posterior)


In [ ]:
#Test emulator error [marginalised]
emu_err = likelihood_instance._get_emulator_error_averaged_mean_flux(test_params)
print('emulator error marginalised =', emu_err)


In [ ]:
#Test emulator error
emu_k, emu_power, emu_std = likelihood_instance.get_predicted(test_params_mf)


In [ ]:
for i in range(len(emu_k)):
    plt.plot(np.log10(emu_k[i]), np.log10(emu_k[i] * emu_power[i] / np.pi))


In [ ]:
for i in range(len(emu_k)):
    plt.plot(np.log10(emu_k[i]), emu_std[i] / emu_power[i])
    plt.plot(np.log10(emu_k[i]), emu_err[i*emu_k[i].size: (i+1)*emu_k[i].size] / emu_power[i], ls=':')


In [ ]:
#Test exploration term
explore = likelihood_instance._get_GP_UCB_exploration_term(emu_err, 13, nu=0.25)
print('Exploration =', explore)


In [ ]:
likelihood_instance.get_data_covariance(-1).shape


In [ ]:
#Test acquisition function
acquire = likelihood_instance.acquisition_function_GP_UCB_marginalised_mean_flux(test_params, nu=0.25,
                                                                                 prior_functions=prior_functions)
print('Acquisition =', acquire)


In [ ]:
#Test acquisition function
acquire_mf = likelihood_instance.acquisition_function_GP_UCB(test_params_mf, nu=0.25, prior_functions=prior_functions)
print('Acquisition =', acquire_mf)


In [ ]:
#Optimise acquisition function
optimise_bounds = [(0.05, 0.95) for i in range(test_params_mf.size)]
optimise_bounds[-1] = (0.05, 1.)
acquire_max = likelihood_instance.optimise_acquisition_function(test_params_mf,
                                                                acquisition_function='GP_UCB',
                                                                optimisation_bounds=optimise_bounds,
                                                                optimisation_method='TNC', nu=0.25,
                                                                prior_functions=prior_functions)


In [ ]:
acquire_max


In [ ]:
acquire_max0 = cp.deepcopy(acquire_max)


In [ ]:
acquire_max0


In [ ]:
#Marginalise mean flux
 fun: -394.72107147587747
     jac: array([ 4.33402533e+01, -3.24205530e+02,  2.16144097e+04,  3.26689360e+00,
        5.59946329e+00, -2.06749746e+00, -9.49059540e+01, -1.35267192e+02,
        2.33754122e+01,  2.24580754e+02,  2.96598682e+02,  6.20065634e+00,
        1.99674491e+02])
 message: 'Converged (|x_n-x_(n-1)| ~= 0)'
    nfev: 93
     nit: 6
  status: 2
 success: True
       x: array([0.54330369, 0.58804541, 0.87045623, 0.45045675, 0.43422056,
       0.406574  , 0.70211524, 0.71994382, 0.67016032, 0.05594524,
       0.05575224, 0.17343253, 0.95731355])


In [ ]:
acquire_max_params = lyl.map_from_unit_cube(acquire_max.x, likelihood_instance.param_limits)


In [ ]:
acquire_max_params = lyl.map_from_unit_cube(acquire_max.x,
                                            likelihood_instance.param_limits[likelihood_instance.zout.shape[0]:])


In [ ]:
print('Maximum of acquisition function =', acquire_max)
print(acquire_max_params)
print('Starting params =', test_params_mf)


In [ ]:
#Plot posteriors
def make_plot(chainfile, savefile, true_parameter_values=None, pnames=None, ranges=None, parameter_indices=None):
    """Make a getdist plot"""
    samples = np.loadtxt(chainfile)
    #A_s hack
    #samples = samples[samples[:, 4] > 2.05e-9, :]

    if parameter_indices is not None:
        samples = samples[:, parameter_indices]
        true_parameter_values = true_parameter_values[parameter_indices]
        pnames = pnames[parameter_indices]
        ranges = ranges[parameter_indices]

    ticks = {}
    if pnames is None:
        #Default emulator parameters
        pnames = [r"d\tau_0", r"\tau_0", r"n_s", r"A_\mathrm{P} \times 10^9", r"H_S", r"H_A", r"h"]
        samples[:,3] *= 1e9
        true_parameter_values[3] *= 1e9
        #Ticks we want to show for each parameter
        ticks = {pnames[3]: [1.5, 2.0, 2.5], pnames[4]: [-0.6,-0.3, 0.], pnames[5]: [0.5,0.7,1.0,1.3], pnames[6]: [0.66, 0.70, 0.74]}
    prange = None
    if ranges is not None:
        prange = {pnames[i] : ranges[i] for i in range(len(pnames))}
    posterior_MCsamples = gd.MCSamples(samples=samples, names=pnames, labels=pnames, label='', ranges=prange)

    print("Sim=",savefile)
    #Get and print the confidence limits
    for i in range(len(pnames)):
        strr = pnames[i]+" 1-sigma, 2-sigma: "
        if i == 6:
            for j in (0.32, 0.05):
                strr += str(round(posterior_MCsamples.confidence(i, j, upper=True),5)) + " "
        else:
            for j in (0.16, 1-0.16, 0.025, 1-0.025):
                strr += str(round(posterior_MCsamples.confidence(i, j),5)) + " "
        print(strr)
    subplot_instance = gdp.getSubplotPlotter()
    subplot_instance.triangle_plot([posterior_MCsamples], filled=True)
#     colour_array = np.array(['black', 'red', 'magenta', 'green', 'green', 'purple', 'turquoise', 'gray', 'red', 'blue'])

    for pi in range(samples.shape[1]):
        for pi2 in range(pi + 1):
            #Place horizontal and vertical lines for the true point
            ax = subplot_instance.subplots[pi, pi2]
            ax.yaxis.label.set_size(24)
            ax.xaxis.label.set_size(24)
            if pi == samples.shape[1]-1 and pnames[pi2] in ticks:
                ax.set_xticks(ticks[pnames[pi2]])
            if pi2 == 0 and pnames[pi] in ticks:
                ax.set_yticks(ticks[pnames[pi]])
            ax.axvline(true_parameter_values[pi2], color='gray', ls='--', lw=4)
            if pi2 < pi:
                ax.axhline(true_parameter_values[pi], color='gray', ls='--', lw=4)
#                #Plot the emulator points
#                 if parameter_index > 1:
#                     ax.scatter(simulation_parameters_latin[:, parameter_index2 - 2], simulation_parameters_latin[:, parameter_index - 2], s=54, color=colour_array[-1], marker='+')
#
#     legend_labels = ['+ Initial Latin hypercube']
#     subplot_instance.add_legend(legend_labels, legend_loc='upper right', colored_text=True, figure=True)
    plt.savefig(savefile)

    return subplot_instance


In [ ]:
chainfile_root = '/home/keir/Plots/nCDM'
chainfile = 'chain_ns0.964As1.83e-09heat_slope0heat_amp1omega_m0.321alpha0beta1gamma-1z_rei8T_rei2e+04_1_emu50_data_TDR_u0_300_ULA_fit_convex_hull_omega_m_fix_Planck_tau_T0_prior_no_jump_Tu0.txt'
chainfile = os.path.join(chainfile_root, chainfile)

subplot_instance = make_plot(chainfile, None, true_parameter_values=test_params_mf,
          pnames=likelihood_instance.likelihood_parameter_names[:, 1],
          ranges=likelihood_instance.param_limits)


In [ ]:
#Plot test points
for a in range(0, test_params_mf.shape[0]):
    for b in range(a + 1):
        ax = subplot_instance.subplots[a, b]
        if b < a:
            ax.scatter(acquire_max_params[b - 0], acquire_max_params[a - 0], s=1000, color='red', marker='+')


In [ ]:
subplot_instance.fig


In [5]:
#BO-1
'''Maximum of acquisition function =      fun: -353.914275727603
     jac: array([ 4.55890307e+02,  4.89668190e+02,  2.95751647e+01,  4.49569450e+02,
       -3.77147819e+02,  1.14626398e+04,  1.19608262e+02,  1.13920612e+02,
       -9.70749170e-01, -7.27263853e+01, -7.31309228e+01,  3.79277651e+01,
        2.01726948e+02,  2.63873142e+02,  5.39544658e+00, -3.13192612e+02])
 message: 'Converged (|x_n-x_(n-1)| ~= 0)'
    nfev: 58
     nit: 8
  status: 2
 success: True
       x: array([0.05      , 0.05      , 0.35450285, 0.60230512, 0.51105329,
       0.87024889, 0.30105636, 0.34192525, 0.40767761, 0.75969048,
       0.73606158, 0.66219663, 0.05      , 0.05      , 0.17386334,
       0.96686619])
[ 7.75000000e-01  7.75000000e-01  9.27251427e-01  9.57218987e-01
  1.86436927e-09  3.20917423e-01  5.73440366e+03  6.19803012e+03
  8.67488662e+03  1.52988653e+00  1.52905577e+00  1.43409983e+00
  2.42028785e+00  3.21983473e+00  7.22534864e+00 -1.90994014e+01]
Starting params = [ 7.7000e-01  7.8000e-01  9.3000e-01  9.6350e-01  1.8296e-09  3.2090e-01
  8.0220e+03  7.6510e+03  8.6730e+03  1.4400e+00  1.4800e+00  1.4500e+00
  4.5600e+00  6.1100e+00  7.2400e+00 -1.9100e+01]'''

acquire_max_params = np.array([ 7.75000000e-01,  7.75000000e-01,  9.27251427e-01,  9.57218987e-01,
  1.86436927e-09,  3.20917423e-01,  5.73440366e+03,  6.19803012e+03,
  8.67488662e+03,  1.52988653e+00,  1.52905577e+00,  1.43409983e+00,
  2.42028785e+00,  3.21983473e+00,  7.22534864e+00, -1.90994014e+01])


In [6]:
#nCDM conversion
nCDM_parameters = likelihood_instance.dark_matter_model(np.array([acquire_max_params[-1],]),
                                                       likelihood_instance.param_limits_nCDM)
print('nCDM parameters =', nCDM_parameters)


nCDM parameters = [ 5.75659790e-03  5.84204677e+00 -1.31882210e+00]


In [13]:
training_parameter_names = np.concatenate((np.array(['ns', 'As', 'omega_m']),
        np.array([key for key in likelihood_instance.emulator.measured_param_names.keys()]),
        np.array(['alpha', 'beta', 'gamma'])))
print(training_parameter_names)


['ns' 'As' 'omega_m' 'T_0_z_5.0' 'T_0_z_4.6' 'T_0_z_4.2' 'gamma_z_5.0'
 'gamma_z_4.6' 'gamma_z_4.2' 'u_0_z_5.0' 'u_0_z_4.6' 'u_0_z_4.2' 'alpha'
 'beta' 'gamma']


In [8]:
#heat_amp
measured_parameters = acquire_max_params[3: -1] #np.array([3, 4, -9, -6, -3])]
measured_parameters = np.concatenate((measured_parameters, nCDM_parameters))

training_parameter_names = np.concatenate((training_parameter_names, np.array(['heat_amp',])))

heat_amp = likelihood_instance.emulator.predict_parameters(measured_parameters,
                                                           training_parameter_names)
#np.array(['ns', 'As', 'T_0_z_4.6', 'gamma_z_4.6',
#            'u_0_z_4.6', 'alpha', 'beta', 'gamma', 'heat_amp']))
print('heat_amp =', heat_amp)


heat_amp = 0.6488485461800885


In [10]:
#heat_slope
#measured_parameters = acquire_max_params[np.array([-6])] #, -3])]
training_parameter_names = np.concatenate((training_parameter_names,
                                           np.array(['heat_slope',])))

heat_slope = likelihood_instance.emulator.predict_parameters(measured_parameters,
                                                             training_parameter_names)
#np.array(['ns', 'As', 'T_0_z_4.6', 'gamma_z_4.6',
#                'u_0_z_4.6', 'alpha', 'beta', 'gamma', 'heat_slope']))
print('heat_slope =', heat_slope)


heat_slope = 0.11017037159243165


In [12]:
#z_rei
#measured_parameters = acquire_max_params[np.array([-9, -6, -3])]
training_parameter_names = np.concatenate((training_parameter_names, np.array(['z_rei',])))

z_rei= likelihood_instance.emulator.predict_parameters(measured_parameters,
                                                       training_parameter_names)
#np.array(['ns', 'As', 'T_0_z_4.6', 'gamma_z_4.6',
#                'u_0_z_4.6', 'alpha', 'beta', 'gamma', 'heat_slope']))
print('z_rei =', z_rei)


z_rei = 10.728634685366917


In [14]:
#T_rei
#measured_parameters = acquire_max_params[np.array([-9, -6, -3])]
training_parameter_names = np.concatenate((training_parameter_names, np.array(['T_rei',])))

T_rei= likelihood_instance.emulator.predict_parameters(measured_parameters,
                                                       training_parameter_names)
#np.array(['ns', 'As', 'T_0_z_4.6', 'gamma_z_4.6',
#                'u_0_z_4.6', 'alpha', 'beta', 'gamma', 'heat_slope']))
print('T_rei =', T_rei)


T_rei = 33156.38537140758


In [ ]:
#Test parameter predictor
colors = ['red', 'blue', 'green']
for i in range(3):
    plt.scatter(likelihood_instance.emulator.measured_sample_params[:, i],
                likelihood_instance.emulator.sample_params[:, 3], color=colors[i])
    plt.axvline(x=measured_parameters[i+3], color=colors[i])
    plt.axhline(y=heat_amp, color=colors[i])
plt.xlabel(r'T_0 [K]')
plt.ylabel(r'heat_amp')


In [ ]:
#Test heat_amp - u_0
for i in range(3):
    plt.scatter(likelihood_instance.emulator.measured_sample_params[:, i+6],
                likelihood_instance.emulator.sample_params[:, 3], color=colors[i])
    plt.axvline(x=measured_parameters[i+9], color=colors[i])
    plt.axhline(y=heat_amp, color=colors[i])
plt.xlabel(r'u_0 [eV/m_p]')
plt.ylabel(r'heat_amp')


In [ ]:
#Test heat_slope
for i in range(3):
    plt.scatter(likelihood_instance.emulator.measured_sample_params[:, i+3],
                likelihood_instance.emulator.sample_params[:, 2], color=colors[i])
    plt.axvline(x=measured_parameters[i+6], color=colors[i])
    plt.axhline(y=heat_slope, color=colors[i])
plt.xlabel(r'gamma (z = 4.6)')
plt.ylabel(r'heat_slope')


In [ ]:
#Test z_rei
for i in range(3):
    plt.scatter(likelihood_instance.emulator.measured_sample_params[:, i+6],
                likelihood_instance.emulator.sample_params[:, 8], color=colors[i])
    plt.axvline(x=measured_parameters[i+9], color=colors[i])
    plt.axhline(y=z_rei, color=colors[i])
plt.xlabel(r'u_0 [eV/m_p]')
plt.ylabel(r'z_rei')


In [ ]:
#Test T_rei
for i in range(3):
    plt.scatter(likelihood_instance.emulator.measured_sample_params[:, i+6],
                likelihood_instance.emulator.sample_params[:, 9], color=colors[i])
    plt.axvline(x=measured_parameters[i+9], color=colors[i])
    plt.axhline(y=T_rei, color=colors[i])
plt.xlabel(r'u_0 [eV/m_p]')
plt.ylabel(r'T_rei [K]')


In [15]:
#Add optimisation simulation
optimise_params = np.concatenate((acquire_max_params[np.array([3, 4])],
                                  np.array([heat_slope, heat_amp]),
                                  np.array([0.3209,]), nCDM_parameters,
                                  np.array([z_rei, T_rei]))).reshape(1, -1)
print('Parameters for optimisation simulation =', optimise_params)


Parameters for optimisation simulation = [[ 9.57218987e-01  1.86436927e-09  1.10170372e-01  6.48848546e-01
   3.20900000e-01  5.75659790e-03  5.84204677e+00 -1.31882210e+00
   1.07286347e+01  3.31563854e+04]]


In [16]:
optimise_params.shape

likelihood_instance.emulator.sample_params.shape[0]


50

In [17]:
likelihood_instance.emulator.gen_simulations(None, 512., 10., samples=optimise_params,
                                             add_optimisation=True)
